# [재배환경 별 작물 종류 예측 - 기계학습 5주차](https://www.kaggle.com/t/c70d8464ca5644ff895eb42b6b26c527)
> 17011520 조영래

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE, ADASYN
import warnings
warnings.filterwarnings(action='ignore')

RANDOM_STATE = 915

In [ ]:
file_path = '/content/drive/MyDrive/2021-1/기계학습(최유경)/5/2021-ml-p3/'

train = pd.read_csv(file_path + 'train.csv')
test = pd.read_csv(file_path + 'test.csv')
sample = pd.read_csv(file_path + 'sample.csv')

## 데이터 살펴보기

In [ ]:
train.shape

(1650, 8)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1650 entries, 0 to 1649
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N            1650 non-null   int64  
 1   P            1650 non-null   int64  
 2   K            1650 non-null   int64  
 3   temperature  1650 non-null   float64
 4   humidity     1650 non-null   float64
 5   ph           1650 non-null   float64
 6   rainfall     1650 non-null   float64
 7   label        1650 non-null   object 
dtypes: float64(4), int64(3), object(1)
memory usage: 103.2+ KB


In [ ]:
train.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,22,36,16,30.581395,50.771481,8.184229,64.585596,mothbeans
1,18,27,41,22.365094,92.308824,7.175344,104.821633,pomegranate
2,61,68,50,35.214628,91.497251,6.793245,243.074507,papaya
3,26,72,22,28.767949,37.577921,4.674942,91.720849,pigeonpeas
4,2,140,197,22.697801,92.822234,5.534567,105.050823,apple


In [ ]:
# 타겟값 균형되게 있는지 확인
train['label'].value_counts()

lentil         85
grapes         81
orange         81
watermelon     79
cotton         79
muskmelon      78
banana         77
maize          77
blackgram      76
kidneybeans    76
rice           75
coffee         74
mungbean       74
mothbeans      74
coconut        73
jute           73
mango          73
apple          71
pomegranate    70
chickpea       69
papaya         69
pigeonpeas     66
Name: label, dtype: int64

In [ ]:
X = train.drop('label', axis=1)
y = train['label']

In [ ]:
y.head()

0      mothbeans
1    pomegranate
2         papaya
3     pigeonpeas
4          apple
Name: label, dtype: object

In [ ]:
y.unique()

array(['mothbeans', 'pomegranate', 'papaya', 'pigeonpeas', 'apple',
       'kidneybeans', 'orange', 'rice', 'mungbean', 'watermelon',
       'coconut', 'jute', 'lentil', 'chickpea', 'cotton', 'grapes',
       'mango', 'muskmelon', 'maize', 'coffee', 'banana', 'blackgram'],
      dtype=object)

In [ ]:
classle = LabelEncoder()
y = classle.fit_transform(y)

print('labels: {}'.format(np.unique(y)))

labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]


In [ ]:
print('inverse transformed labels: {}'\
      .format(np.unique(classle.inverse_transform(y))))

inverse transformed labels: ['apple' 'banana' 'blackgram' 'chickpea' 'coconut' 'coffee' 'cotton'
 'grapes' 'jute' 'kidneybeans' 'lentil' 'maize' 'mango' 'mothbeans'
 'mungbean' 'muskmelon' 'orange' 'papaya' 'pigeonpeas' 'pomegranate'
 'rice' 'watermelon']


In [ ]:
classle.classes_

array(['apple', 'banana', 'blackgram', 'chickpea', 'coconut', 'coffee',
       'cotton', 'grapes', 'jute', 'kidneybeans', 'lentil', 'maize',
       'mango', 'mothbeans', 'mungbean', 'muskmelon', 'orange', 'papaya',
       'pigeonpeas', 'pomegranate', 'rice', 'watermelon'], dtype=object)

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size = 0.3,
                                                  random_state = RANDOM_STATE,
                                                  stratify = y)

In [ ]:
print(X_train.shape)
print(X_validation.shape)
print(y_train.shape)
print(y_validation.shape)

(1155, 7)
(495, 7)
(1155,)
(495,)


## 하이퍼파라미터 조정 없이 예측 확인

In [ ]:
knn_clf_n5_p2 = KNeighborsClassifier(n_neighbors=5, p=2)
knn_clf_n5_p2.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
y_train_pred = knn_clf_n5_p2.predict(X_train)
y_validation_pred = knn_clf_n5_p2.predict(X_validation)

print('Misclassified training samples: {}'.format((y_train != y_train_pred).sum()))
print('Misclassified validation samples: {}'.format((y_validation != y_validation_pred).sum()))

Misclassified training samples: 9
Misclassified validation samples: 12


In [ ]:
print('training accuracy_score: {}'.format(accuracy_score(y_train, y_train_pred)))
print('validation accuracy_score: {}'.format(accuracy_score(y_validation, y_validation_pred)))

training accuracy_score: 0.9922077922077922
validation accuracy_score: 0.9757575757575757


In [ ]:
conf = confusion_matrix(y_true = y_validation, y_pred = y_validation_pred)
conf

array([[21,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 0, 23,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 0,  0, 23,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 21,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 24,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 24,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 21,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 23,  0,  0,

In [ ]:
std_scaler = StandardScaler()
std_scaler.fit(X_train)

X_train_std = std_scaler.transform(X_train)
X_validation_std = std_scaler.transform(X_validation)

print(X_train.head())
print(X_train_std[:5, ])

       N   P   K  temperature   humidity        ph    rainfall
905   34  34  35    27.274332  47.168081  6.422711   95.257992
803   75  54  36    26.294655  84.569193  7.023936  257.491491
306  110  28  46    24.291050  88.045413  6.498896   51.260464
568    9  60  23    31.969879  57.173770  6.276004   64.255204
835   26  10  33    28.272981  96.936495  6.070718  198.823486
[[-0.46106949 -0.59397215 -0.25657785  0.34667092 -1.11375879 -0.05705504
  -0.12809544]
 [ 0.64327251  0.0135711  -0.23698205  0.1523712   0.59054348  0.7186917
   2.81920992]
 [ 1.58600349 -0.77623513 -0.04102395 -0.24500438  0.74894866  0.04124497
  -0.92740108]
 [-1.13444876  0.19583407 -0.49172756  1.27794035 -0.65781728 -0.2463464
  -0.69132489]
 [-0.67655086 -1.32302405 -0.29576947  0.54473343  1.15409943 -0.51122242
   1.75338489]]


In [ ]:
# 표준화된 데이터가 더 안 좋은 성능이 나옴

knn_clf_n5_p2_std = KNeighborsClassifier(n_neighbors=5, p=2)
knn_clf_n5_p2_dstd.fit(X_train_std, y_train)

y_train_std_pred = knn_clf_n5_p2_std.predict(X_train_std)
y_validation_std_pred = knn_clf_n5_p2_std.predict(X_validation_std)

print('Misclassified training samples: {}'.format((y_train != y_train_std_pred).sum()))
print('Misclassified validation samples: {}'.format((y_validation != y_validation_std_pred).sum()))

print('training accuracy_score: {}'.format(accuracy_score(y_train, y_train_std_pred)))
print('validation accuracy_score: {}'.format(accuracy_score(y_validation, y_validation_std_pred)))

Misclassified training samples: 13
Misclassified validation samples: 15
training accuracy_score: 0.9887445887445887
validation accuracy_score: 0.9696969696969697


In [ ]:
# 가중평균을 이용해봄 -> validation셋에 대해서는 같으나 train셋에 대해서는 조금 더 좋다 가중평균을 이용해봄 -> validation셋에 대해서는 같으나 train셋에 대해서는 조금 더 좋다

knn_clf_n5_p2_dist = KNeighborsClassifier(n_neighbors=5, p=2, weights='distance')
knn_clf_n5_p2_dist.fit(X_train, y_train)

y_train_pred = knn_clf_n5_p2_dist.predict(X_train)
y_validation_pred = knn_clf_n5_p2_dist.predict(X_validation)

print('Misclassified training samples: {}'.format((y_train != y_train_pred).sum()))
print('Misclassified validation samples: {}'.format((y_validation != y_validation_pred).sum()))

print('training accuracy_score: {}'.format(accuracy_score(y_train, y_train_pred)))
print('validation accuracy_score: {}'.format(accuracy_score(y_validation, y_validation_pred)))

Misclassified training samples: 0
Misclassified validation samples: 12
training accuracy_score: 1.0
validation accuracy_score: 0.9757575757575757


## 처리되지 않은 데이터로 확인

In [ ]:
# KNeiborsClassifier 의 하이퍼파라미터를 탐색하는 함수

def knn_clf_search_params(X_train, X_test, y_train, y_test, verbose=True):
  weights = ['uniform', 'distance']
  p = [1, 2]
  n_neighbors = [1, 2, 3, 5, 7, 10, 15, 20]

  accuracy_scores_with_params = []

  for weights_ in weights:
    for p_ in p:
      for n_neighbors_ in n_neighbors:
        knn_clf = KNeighborsClassifier(weights=weights_,
                                      p=p_, n_neighbors=n_neighbors_)
        knn_clf.fit(X_train, y_train)

        y_train_pred = knn_clf.predict(X_train)
        y_test_pred = knn_clf.predict(X_test)

        accuracy_score_train = accuracy_score(y_train, y_train_pred)
        accuracy_score_validation = accuracy_score(y_test, y_test_pred)

        if verbose:
          print(f'##### weights: {weights_}, p: {p_}, n_neighbors: {n_neighbors_}')
          print(f'training accuracy_score: {accuracy_score_train}')
          print(f'validation accuracy_score: {accuracy_score_validation}')
          print('\n')

        accuracy_scores_with_params.append((accuracy_score_train,
                                            accuracy_score_validation,
                                            (weights_, p_, n_neighbors_)))
        
  return accuracy_scores_with_params

In [ ]:
accuracy_scores_with_params = knn_clf_search_params(
    X_train, X_validation, y_train, y_validation
)

##### weights: uniform, p: 1, n_neighbors: 1
training accuracy_score: 1.0
validation accuracy_score: 0.98989898989899


##### weights: uniform, p: 1, n_neighbors: 2
training accuracy_score: 0.9896103896103896
validation accuracy_score: 0.9777777777777777


##### weights: uniform, p: 1, n_neighbors: 3
training accuracy_score: 0.9904761904761905
validation accuracy_score: 0.9818181818181818


##### weights: uniform, p: 1, n_neighbors: 5
training accuracy_score: 0.9904761904761905
validation accuracy_score: 0.9838383838383838


##### weights: uniform, p: 1, n_neighbors: 7
training accuracy_score: 0.9913419913419913
validation accuracy_score: 0.9858585858585859


##### weights: uniform, p: 1, n_neighbors: 10
training accuracy_score: 0.9826839826839827
validation accuracy_score: 0.9757575757575757


##### weights: uniform, p: 1, n_neighbors: 15
training accuracy_score: 0.9818181818181818
validation accuracy_score: 0.9757575757575757


##### weights: uniform, p: 1, n_neighbors: 20
training a

In [ ]:
# train accuracy, validation accuracy, (weights, p, n_neighbors) 순서로 출력
# train accuracy 내림차순 정렬하여 10개 출력

for accuracy_params in sorted(accuracy_scores_with_params,
                              key= lambda x: x[0], reverse=True)[:10]:
  print(accuracy_params)

(1.0, 0.98989898989899, ('uniform', 1, 1))
(1.0, 0.9838383838383838, ('uniform', 2, 1))
(1.0, 0.98989898989899, ('distance', 1, 1))
(1.0, 0.98989898989899, ('distance', 1, 2))
(1.0, 0.9818181818181818, ('distance', 1, 3))
(1.0, 0.9858585858585859, ('distance', 1, 5))
(1.0, 0.9858585858585859, ('distance', 1, 7))
(1.0, 0.9878787878787879, ('distance', 1, 10))
(1.0, 0.9818181818181818, ('distance', 1, 15))
(1.0, 0.9818181818181818, ('distance', 1, 20))


In [ ]:
# validation accuracy 내림차순 정렬하여 10개 출력

for accuracy_params in sorted(accuracy_scores_with_params,
                              key= lambda x: x[1], reverse=True)[:10]:
  print(accuracy_params)

(1.0, 0.98989898989899, ('uniform', 1, 1))
(1.0, 0.98989898989899, ('distance', 1, 1))
(1.0, 0.98989898989899, ('distance', 1, 2))
(1.0, 0.9878787878787879, ('distance', 1, 10))
(0.9913419913419913, 0.9858585858585859, ('uniform', 1, 7))
(1.0, 0.9858585858585859, ('distance', 1, 5))
(1.0, 0.9858585858585859, ('distance', 1, 7))
(0.9904761904761905, 0.9838383838383838, ('uniform', 1, 5))
(1.0, 0.9838383838383838, ('uniform', 2, 1))
(1.0, 0.9838383838383838, ('distance', 2, 1))


In [ ]:
# final_model = KNeighborsClassifier(weights='distance', p=1, n_neighbors=2)  # 0.96545
# final_model = KNeighborsClassifier(weights='distance', p=1, n_neighbors=10) # 0.97272
# final_model = KNeighborsClassifier(weights='distance', p=1, n_neighbors=5)  # 0.97090
# final_model = KNeighborsClassifier(weights='uniform', p=1, n_neighbors=1)  # 0.96545
final_model = KNeighborsClassifier(weights='distance', p=1, n_neighbors=7)  # 0.97454

final_model.fit(X, y)
y_pred = final_model.predict(test)
y_pred = classle.inverse_transform(y_pred)

sample['label'] = y_pred
sample.to_csv('submission.csv', index=False)

## 표준화된 데이터로 확인

In [ ]:
accuracy_scores_with_params = knn_clf_search_params(
    X_train_std, X_validation_std, y_train, y_validation, verbose=False
)

In [ ]:
# train accuracy, validation accuracy, (weights, p, n_neighbors) 순서로 출력
# train accuracy 내림차순 정렬하여 10개 출력

for accuracy_params in sorted(accuracy_scores_with_params,
                              key= lambda x: x[0], reverse=True)[:10]:
  print(accuracy_params)

(1.0, 0.9757575757575757, ('uniform', 1, 1))
(1.0, 0.9797979797979798, ('uniform', 2, 1))
(1.0, 0.9757575757575757, ('distance', 1, 1))
(1.0, 0.9757575757575757, ('distance', 1, 2))
(1.0, 0.9696969696969697, ('distance', 1, 3))
(1.0, 0.9737373737373738, ('distance', 1, 5))
(1.0, 0.9696969696969697, ('distance', 1, 7))
(1.0, 0.9676767676767677, ('distance', 1, 10))
(1.0, 0.9717171717171718, ('distance', 1, 15))
(1.0, 0.9636363636363636, ('distance', 1, 20))


In [ ]:
# validation accuracy 내림차순 정렬하여 10개 출력

for accuracy_params in sorted(accuracy_scores_with_params,
                              key= lambda x: x[1], reverse=True)[:10]:
  print(accuracy_params)

(1.0, 0.9797979797979798, ('uniform', 2, 1))
(1.0, 0.9797979797979798, ('distance', 2, 1))
(1.0, 0.9797979797979798, ('distance', 2, 2))
(1.0, 0.9757575757575757, ('uniform', 1, 1))
(0.9948051948051948, 0.9757575757575757, ('uniform', 2, 3))
(1.0, 0.9757575757575757, ('distance', 1, 1))
(1.0, 0.9757575757575757, ('distance', 1, 2))
(1.0, 0.9757575757575757, ('distance', 2, 3))
(1.0, 0.9737373737373738, ('distance', 1, 5))
(0.9913419913419913, 0.9717171717171718, ('uniform', 1, 5))


In [ ]:
X_std = std_scaler.fit_transform(X)
test_std = std_scaler.transform(test)

In [ ]:
# final_model_std = KNeighborsClassifier(weights='distance', p=2, n_neighbors=3)  # 0.96909
# final_model_std = KNeighborsClassifier(weights='uniform', p=2, n_neighbors=3) # 0.96545
final_model_std = KNeighborsClassifier(weights='distance', p=1, n_neighbors=5) # 0.97090

final_model_std.fit(X_std, y)
y_pred = final_model_std.predict(test_std)
y_pred = classle.inverse_transform(y_pred)

sample['label'] = y_pred
sample.to_csv('submission_std.csv', index=False)

## 정규화된 데이터로 확인

In [ ]:
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(X_train)

X_train_minmax = std_scaler.transform(X_train)
X_validation_minmax = std_scaler.transform(X_validation)

In [ ]:
accuracy_scores_with_params = knn_clf_search_params(
    X_train_minmax, X_validation_minmax, y_train, y_validation, verbose=False
)

In [ ]:
# validation accuracy 내림차순 정렬하여 10개 출력

for accuracy_params in sorted(accuracy_scores_with_params,
                              key= lambda x: x[1], reverse=True)[:10]:
  print(accuracy_params)

(1.0, 0.9797979797979798, ('uniform', 2, 1))
(1.0, 0.9797979797979798, ('distance', 2, 1))
(1.0, 0.9797979797979798, ('distance', 2, 2))
(1.0, 0.9757575757575757, ('uniform', 1, 1))
(0.9948051948051948, 0.9757575757575757, ('uniform', 2, 3))
(1.0, 0.9757575757575757, ('distance', 1, 1))
(1.0, 0.9757575757575757, ('distance', 1, 2))
(1.0, 0.9757575757575757, ('distance', 2, 3))
(1.0, 0.9737373737373738, ('distance', 1, 5))
(0.9922077922077922, 0.9717171717171718, ('uniform', 1, 5))


In [ ]:
X_minmax = min_max_scaler.fit_transform(X)
test_minmax = min_max_scaler.transform(test)

In [ ]:
# final_model_minmax = KNeighborsClassifier(weights='distance', p=1, n_neighbors=5) # 0.97272
# final_model_minmax = KNeighborsClassifier(weights='distance', p=2, n_neighbors=3) # 0.97090
# final_model_minmax = KNeighborsClassifier(weights='uniform', p=2, n_neighbors=3) # 0.97090
final_model_minmax = KNeighborsClassifier(weights='distance', p=2, n_neighbors=2) # 0.96909

final_model_minmax.fit(X_minmax, y)
y_pred = final_model_minmax.predict(test_minmax)
y_pred = classle.inverse_transform(y_pred)

sample['label'] = y_pred
sample.to_csv('submission_minmax.csv', index=False)

## scaled된 데이터로 확인한 결론
- scaling 하지 않는 것이 가장 잘 나옴

## OverSampling 한 데이터로 확인해보기
> 엄청 불균형하지는 않지만 한 번 시도

### SMOTE

In [ ]:
smote = SMOTE(random_state=RANDOM_STATE)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
X_validation_smote, y_validation_smote = smote.fit_resample(X_validation, y_validation)

In [ ]:
print(X_train.shape)
print(X_train_smote.shape)
print(X_validation.shape)
print(X_validation_smote.shape)

(1155, 7)
(1298, 7)
(495, 7)
(572, 7)


In [ ]:
print(y_train.shape)
print(y_train_smote.shape)
print(y_validation.shape)
print(y_validation_smote.shape)

(1155,)
(1298,)
(495,)
(572,)


In [ ]:
accuracy_scores_with_params = knn_clf_search_params(
    X_train_smote, X_validation_smote, y_train_smote, y_validation_smote, verbose=False
)

In [ ]:
# validation accuracy 내림차순 정렬하여 10개 출력

for accuracy_params in sorted(accuracy_scores_with_params,
                              key= lambda x: x[1], reverse=True)[:10]:
  print(accuracy_params)

(1.0, 0.9895104895104895, ('uniform', 1, 1))
(0.9915254237288136, 0.9895104895104895, ('uniform', 1, 2))
(0.9915254237288136, 0.9895104895104895, ('uniform', 1, 3))
(1.0, 0.9895104895104895, ('distance', 1, 1))
(1.0, 0.9895104895104895, ('distance', 1, 2))
(1.0, 0.9895104895104895, ('distance', 1, 3))
(0.9922958397534669, 0.9877622377622378, ('uniform', 1, 7))
(1.0, 0.9877622377622378, ('distance', 1, 7))
(0.9922958397534669, 0.986013986013986, ('uniform', 1, 5))
(1.0, 0.986013986013986, ('uniform', 2, 1))


In [ ]:
X_smote, y_smote = smote.fit_resample(X, y)

In [ ]:
# final_model_smote = KNeighborsClassifier(weights='distance', p=1, n_neighbors=3) # 0.97090
# final_model_smote = KNeighborsClassifier(weights='distance', p=1, n_neighbors=5) # 0.97090
final_model_smote = KNeighborsClassifier(weights='distance', p=1, n_neighbors=7) # 0.97454

final_model_smote.fit(X_smote, y_smote)
y_pred = final_model_smote.predict(test)
y_pred = classle.inverse_transform(y_pred)

sample['label'] = y_pred
sample.to_csv('submission_smote.csv', index=False)

## ADASYN
> 오류 발생 SMOTE 사용할 것을 권장

```python
adasyn = ADASYN(random_state=RANDOM_STATE)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train, y_train)
X_validation_adasyn, y_validation_adasyn = adasyn.fit_resample(X_validation, y_validation)
```
```
RuntimeError                              Traceback (most recent call last)
<ipython-input-62-59a8ba8e9c7d> in <module>()
      1 adasyn = ADASYN(random_state=RANDOM_STATE)
----> 2 X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train, y_train)
      3 X_validation_adasyn, y_validation_adasyn = adasyn.fit_resample(X_validation, y_validation)

1 frames
/usr/local/lib/python3.7/dist-packages/imblearn/base.py in fit_resample(self, X, y)
     83             self.sampling_strategy, y, self._sampling_type)
     84 
---> 85         output = self._fit_resample(X, y)
     86 
     87         if binarize_y:

/usr/local/lib/python3.7/dist-packages/imblearn/over_sampling/_adasyn.py in _fit_resample(self, X, y)
    123                         (self.nn_.n_neighbors - 1))
    124             if not np.sum(ratio_nn):
--> 125                 raise RuntimeError('Not any neigbours belong to the majority'
    126                                    ' class. This case will induce a NaN case'
    127                                    ' with a division by zero. ADASYN is not'

RuntimeError: Not any neigbours belong to the majority class. This case will induce a NaN case with a division by zero. ADASYN is not suited for this specific dataset. Use SMOTE instead.
```


## oversampling도 크게 성능 향상이 이뤄지지 않는 것 같다.